# Assignment 2
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    cities["NHL"]=cities["NHL"].str.replace(r"([A-Z])",r" \1",regex=True)
    cities["NHL"]=cities["NHL"].str.replace(r"\[.+\]","",regex=True)
    citie_nhl=cities["NHL"].str.split().explode()
    
    citie_nhl=citie_nhl.to_frame(name="namenhl")
    citie_nhl=citie_nhl.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
    nhl_wd=nhl_df[nhl_df["team"]!=nhl_df["GP"]]
    nhl_wd=nhl_wd[nhl_wd["year"]==2018]

    nhl_wd["W"] = nhl_wd["W"].astype(int)
    nhl_wd["L"] = nhl_wd["L"].astype(int)
    nhl_wd["ratio_nhl"]=nhl_wd["W"]/(nhl_wd["L"]+nhl_wd["W"])

    nhl_wd["team"]=nhl_wd["team"].str.replace('*', '',regex=True)
    nhl_wd["namenhl"]=nhl_wd["team"].str.split().str[-1]
    nhl_wd=nhl_wd.merge(citie_nhl,on="namenhl",how="left")
    nhl_wd=nhl_wd.sort_values("Metropolitan area")
    nhl_wd["Population"]=nhl_wd["Population (2016 est.)[8]"].astype(int)
    grouped_nhl=nhl_wd.groupby(["Metropolitan area"])["ratio_nhl"].mean()
    grouped_nhl=grouped_nhl.to_frame()
    grouped_nhl=grouped_nhl.merge(nhl_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
    grouped_nhl=grouped_nhl.drop_duplicates(subset="Metropolitan area")
    #raise NotImplementedError()
    
    population_by_region = grouped_nhl["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped_nhl["ratio_nhl"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nhl_correlation()

(0.012486162921209909, 0.9497182859911791)

In [ ]:
cities

In [ ]:
# Result like that
(0.012486162921209881, 0.9497182859911791)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    cities["NBA"]=cities["NBA"].str.replace(r"([A-Z])",r" \1",regex=True)
    cities["NBA"]=cities["NBA"].str.replace(r"\[.+\]","",regex=True)
    citie=cities["NBA"].str.split().explode()
    citie=citie.to_frame(name="nba_team")
    citie=citie.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
    nba_wd=nba[nba["team"]!=nba["GB"]]
    nba_wd=nba_wd[nba_wd["year"]==2018]
    nba_wd["W"] = nba_wd["W"].astype(int)
    nba_wd["L"] = nba_wd["L"].astype(int)
    nba_wd["ratio_nba"]=nba_wd["W"]/(nba_wd["L"]+nba_wd["W"])
    nba_wd["team"]=nba_wd["team"].str.replace('*', '',regex=True)
    nba_wd["team"]=nba_wd["team"].str.replace(r"\(.*\)", "",regex=True)
    nba_wd["nba_team"]=nba_wd["team"].str.split().str[-1]
    nba_wd=nba_wd.merge(citie,on="nba_team",how="left")

    nba_wd["Population"]=nba_wd["Population (2016 est.)[8]"].astype(float)
    grouped_nba=nba_wd.groupby(["Metropolitan area"])["ratio_nba"].mean()
    grouped_nba=grouped_nba.to_frame()
    grouped_nba=grouped_nba.merge(nba_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
    grouped_nba=grouped_nba.drop_duplicates(subset="Metropolitan area")
    #raise NotImplementedError()
    
    population_by_region = grouped_nba["Population"] # pass in metropolitan area population from cities
    win_loss_by_region =   grouped_nba["ratio_nba"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
nba_correlation()

(-0.17657160252844614, 0.36874741604463)

In [ ]:
# Result like that
-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
mlb_df=pd.read_csv("assets/mlb.csv")
#print(mlb_df.head())
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities["MLB"]=cities["MLB"].str.replace(r"([A-Z])",r" \1",regex=True)
cities["MLB"]=cities["MLB"].str.replace(r"\[.+\]","",regex=True)
citie_mlb=cities["MLB"].str.split().explode()
citie_mlb=citie_mlb.to_frame(name="namemlb")
citie_mlb=citie_mlb.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)

mlb_wd=mlb_df[mlb_df["team"]!=mlb_df["W"]]
mlb_wd=mlb_wd[mlb_wd["year"]==2018]
mlb_wd["W"] = mlb_wd["W"].astype(int)
mlb_wd["L"] = mlb_wd["L"].astype(int)
mlb_wd["ratio_mlb"]=mlb_wd["W"]/(mlb_wd["L"]+mlb_wd["W"])
mlb_wd["team"]=mlb_wd["team"].str.replace('*', '',regex=True)
mlb_wd["namemlb"]=mlb_wd["team"].str.split().str[-1]

mlb_wd=mlb_wd.merge(citie_mlb,on="namemlb",how="left")
mlb_wd=mlb_wd.drop([0,10])

mlb_wd["Population"]=mlb_wd["Population (2016 est.)[8]"].astype(int)
grouped_mlb=mlb_wd.groupby(["Metropolitan area"])["ratio_mlb"].mean()
grouped_mlb=grouped_mlb.to_frame()
grouped_mlb=grouped_mlb.merge(mlb_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
grouped_mlb=grouped_mlb.drop_duplicates(subset="Metropolitan area")
#print(grouped_mlb)
#type(grouped_mlb)
print(stats.pearsonr(grouped_mlb["ratio_mlb"],grouped_mlb["Population"]))

(0.15027698302669307, 0.46370703378875583)


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    cities["MLB"]=cities["MLB"].str.replace(r"([A-Z])",r" \1",regex=True)
    cities["MLB"]=cities["MLB"].str.replace(r"\[.+\]","",regex=True)
    citie_mlb=cities["MLB"].str.split().explode()
    citie_mlb=citie_mlb.to_frame(name="namemlb")
    citie_mlb=citie_mlb.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)

    mlb_wd=mlb_df[mlb_df["team"]!=mlb_df["W"]]
    mlb_wd=mlb_wd[mlb_wd["year"]==2018]
    mlb_wd["W"] = mlb_wd["W"].astype(int)
    mlb_wd["L"] = mlb_wd["L"].astype(int)
    mlb_wd["ratio_mlb"]=mlb_wd["W"]/(mlb_wd["L"]+mlb_wd["W"])
    mlb_wd["team"]=mlb_wd["team"].str.replace('*', '',regex=True)
    mlb_wd["namemlb"]=mlb_wd["team"].str.split().str[-1]

    mlb_wd=mlb_wd.merge(citie_mlb,on="namemlb",how="left")
    mlb_wd=mlb_wd.drop([0,10])

    mlb_wd["Population"]=mlb_wd["Population (2016 est.)[8]"].astype(int)
    grouped_mlb=mlb_wd.groupby(["Metropolitan area"])["ratio_mlb"].mean()
    grouped_mlb=grouped_mlb.to_frame()
    grouped_mlb=grouped_mlb.merge(mlb_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
    grouped_mlb=grouped_mlb.drop_duplicates(subset="Metropolitan area")
    #raise NotImplementedError()
    
    population_by_region = grouped_mlb["Population"] # pass in metropolitan area population from cities
    win_loss_by_region   = grouped_mlb["ratio_mlb"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)
mlb_correlation()

In [ ]:
# Result like that
0.14213659322560204

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    cities["NFL"]=cities["NFL"].str.replace(r"([A-Z])",r" \1",regex=True)
    cities["NFL"]=cities["NFL"].str.replace(r"\[.+\]","",regex=True)
    citie_nfl=cities["NFL"].str.split().explode()
    citie_nfl=citie_nfl.to_frame(name="namenfl")
    citie_nfl=citie_nfl.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
    nfl_wd=nfl[nfl["team"]!=nfl["W"]]
    nfl_wd=nfl_wd[nfl_wd["year"]==2018]
    nfl_wd["W"] = nfl_wd["W"].astype(int)
    nfl_wd["L"] = nfl_wd["L"].astype(int)
    nfl_wd["ratio_nfl"]=nfl_wd["W"]/(nfl_wd["L"]+nfl_wd["W"])
    nfl_wd["team"]=nfl_wd["team"].str.replace('*', '',regex=True)
    nfl_wd["team"]=nfl_wd["team"].str.replace('+', '',regex=True)
    nfl_wd["namenfl"]=nfl_wd["team"].str.split().str[-1]
    nfl_wd=nfl_wd.merge(citie_nfl,on="namenfl",how="left")
    nfl_wd=nfl_wd.sort_values("namenfl")
    
    nfl_wd["Population"]=nfl_wd["Population (2016 est.)[8]"].astype(int)
    grouped_nfl=nfl_wd.groupby(["Metropolitan area"])["ratio_nfl"].mean()
    grouped_nfl=grouped_nfl.to_frame()
    grouped_nfl=grouped_nfl.merge(nfl_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
    grouped_nfl=grouped_nfl.drop_duplicates(subset="Metropolitan area")
    #raise NotImplementedError()
    
    population_by_region = grouped_nfl["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped_nfl["ratio_nfl"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nfl_correlation()

(0.004922112149349409, 0.9797833458363694)

In [ ]:
# Result like that
(0.00894077067749839, 0.9632859803293105)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb=pd.read_csv("assets/mlb.csv")
nhl=pd.read_csv("assets/nhl.csv")
nba=pd.read_csv("assets/nba.csv")
nfl=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities["NHL"]=cities["NHL"].str.replace(r"([A-Z])",r" \1",regex=True)
cities["NHL"]=cities["NHL"].str.replace(r"\[.+\]","",regex=True)
citie_nhl=cities["NHL"].str.split().explode()    
citie_nhl=citie_nhl.to_frame(name="namenhl")
citie_nhl=citie_nhl.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
nhl_wd=nhl[nhl["team"]!=nhl["GP"]]
nhl_wd=nhl_wd[nhl_wd["year"]==2018]
nhl_wd["W"] = nhl_wd["W"].astype(int)
nhl_wd["L"] = nhl_wd["L"].astype(int)
nhl_wd["ratio_nhl"]=nhl_wd["W"]/(nhl_wd["L"]+nhl_wd["W"])
nhl_wd["team"]=nhl_wd["team"].str.replace('*', '',regex=True)
nhl_wd["namenhl"]=nhl_wd["team"].str.split().str[-1]
nhl_wd=nhl_wd.merge(citie_nhl,on="namenhl",how="left")
nhl_wd=nhl_wd.sort_values("Metropolitan area")
nhl_wd["Population"]=nhl_wd["Population (2016 est.)[8]"].astype(int)
grouped_nhl=nhl_wd.groupby(["Metropolitan area"])["ratio_nhl"].mean()
grouped_nhl=grouped_nhl.to_frame()
grouped_nhl=grouped_nhl.merge(nhl_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
grouped_nhl=grouped_nhl.drop_duplicates(subset="Metropolitan area")

cities["NBA"]=cities["NBA"].str.replace(r"([A-Z])",r" \1",regex=True)
cities["NBA"]=cities["NBA"].str.replace(r"\[.+\]","",regex=True)
citie=cities["NBA"].str.split().explode()
citie=citie.to_frame(name="nba_team")
citie=citie.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
nba_wd=nba[nba["team"]!=nba["GB"]]
nba_wd=nba_wd[nba_wd["year"]==2018]
nba_wd["W"] = nba_wd["W"].astype(int)
nba_wd["L"] = nba_wd["L"].astype(int)
nba_wd["ratio_nba"]=nba_wd["W"]/(nba_wd["L"]+nba_wd["W"])
nba_wd["team"]=nba_wd["team"].str.replace('*', '',regex=True)
nba_wd["team"]=nba_wd["team"].str.replace(r"\(.*\)", "",regex=True)
nba_wd["nba_team"]=nba_wd["team"].str.split().str[-1]
nba_wd=nba_wd.merge(citie,on="nba_team",how="left")
nba_wd["Population"]=nba_wd["Population (2016 est.)[8]"].astype(float)
grouped_nba=nba_wd.groupby(["Metropolitan area"])["ratio_nba"].mean()
grouped_nba=grouped_nba.to_frame()
grouped_nba=grouped_nba.merge(nba_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
grouped_nba=grouped_nba.drop_duplicates(subset="Metropolitan area")

cities["MLB"]=cities["MLB"].str.replace(r"([A-Z])",r" \1",regex=True)
cities["MLB"]=cities["MLB"].str.replace(r"\[.+\]","",regex=True)
citie_mlb=cities["MLB"].str.split().explode()
citie_mlb=citie_mlb.to_frame(name="namemlb")
citie_mlb=citie_mlb.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
mlb_wd=mlb_df[mlb_df["team"]!=mlb_df["W"]]
mlb_wd=mlb_wd[mlb_wd["year"]==2018]
mlb_wd["W"] = mlb_wd["W"].astype(int)
mlb_wd["L"] = mlb_wd["L"].astype(int)
mlb_wd["ratio_mlb"]=mlb_wd["W"]/(mlb_wd["L"]+mlb_wd["W"])
mlb_wd["team"]=mlb_wd["team"].str.replace('*', '',regex=True)
mlb_wd["namemlb"]=mlb_wd["team"].str.split().str[-1]
mlb_wd=mlb_wd.merge(citie_mlb,on="namemlb",how="left")
mlb_wd=mlb_wd.drop([0,10])
mlb_wd["Population"]=mlb_wd["Population (2016 est.)[8]"].astype(int)
grouped_mlb=mlb_wd.groupby(["Metropolitan area"])["ratio_mlb"].mean()
grouped_mlb=grouped_mlb.to_frame()
grouped_mlb=grouped_mlb.merge(mlb_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
grouped_mlb=grouped_mlb.drop_duplicates(subset="Metropolitan area")

cities["NFL"]=cities["NFL"].str.replace(r"([A-Z])",r" \1",regex=True)
cities["NFL"]=cities["NFL"].str.replace(r"\[.+\]","",regex=True)
citie_nfl=cities["NFL"].str.split().explode()
citie_nfl=citie_nfl.to_frame(name="namenfl")
citie_nfl=citie_nfl.merge(cities[["Metropolitan area","Population (2016 est.)[8]"]],left_index=True, right_index=True)
nfl_wd=nfl[nfl["team"]!=nfl["W"]]
nfl_wd=nfl_wd[nfl_wd["year"]==2018]
nfl_wd["W"] = nfl_wd["W"].astype(int)
nfl_wd["L"] = nfl_wd["L"].astype(int)
nfl_wd["ratio_nfl"]=nfl_wd["W"]/(nfl_wd["L"]+nfl_wd["W"])
nfl_wd["team"]=nfl_wd["team"].str.replace('*', '',regex=True)
nfl_wd["team"]=nfl_wd["team"].str.replace('+', '',regex=True)
nfl_wd["namenfl"]=nfl_wd["team"].str.split().str[-1]
nfl_wd=nfl_wd.merge(citie_nfl,on="namenfl",how="left")
nfl_wd=nfl_wd.sort_values("namenfl") 
nfl_wd["Population"]=nfl_wd["Population (2016 est.)[8]"].astype(int)
grouped_nfl=nfl_wd.groupby(["Metropolitan area"])["ratio_nfl"].mean()
grouped_nfl=grouped_nfl.to_frame()
grouped_nfl=grouped_nfl.merge(nfl_wd[["Population","Metropolitan area"]],on="Metropolitan area",how="left")
grouped_nfl=grouped_nfl.drop_duplicates(subset="Metropolitan area")

def sports_team_performance():
    # YOUR CODE HERE
    grouped_all=grouped_nhl.merge(grouped_nba,on="Metropolitan area",how="outer")
    grouped_all=grouped_all.merge(grouped_mlb,on="Metropolitan area",how="outer")
    grouped_all=grouped_all.merge(grouped_nfl,on="Metropolitan area",how="outer")
    grouped_all=grouped_all.drop(columns=["Population_x","Population_y"],axis=1)
    grouped_all=grouped_all.drop(columns="Metropolitan area",axis=1)
    grouped_all.columns = ['NHL', 'NBA', 'MLB', 'NFL']
    print(grouped_all)
    pvs=pd.DataFrame(columns=grouped_all.columns,index=grouped_all.columns)
    
    for i in range(0,4):
        df=grouped_all
        for j in range(0,4):
            df_ij=df.iloc[:,[i,j]]
            df_ij=df_ij.dropna()
            pvs.iloc[i,j]=stats.ttest_rel(df_ij.iloc[:,0],df_ij.iloc[:,1])[1]
            
    p_values=pvs
    
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

In [ ]:
# Result like that
import pandas as pd
result=pd.read_csv('result.csv',index_col='Unnamed: 0')
result